# Project requirements& prerequisites

### python3.0+
### keras using tensorflow backends
### glob, pandas, numpy for data processing
### music21 for computer musicology

### Using Google Colab as compiler(optional)

In [0]:
!pip install music21

### Parsing midi files from Google Drive and convert all the information needed properly (mostly into list) and save  them as .csv files for further uses

Here intend to make the model easier to train, we choose to transpose all the midi files (and all the information within them like notes and chords) to the same key (Cmaj or Amin).

We treat the midi stream in any single measure as a "sentence". And we put all the "sentences" together and save them.

In [0]:
# -*- coding: UTF-8 -*-
from google.colab import drive
drive.mount('/gdrive')
from music21 import midi, interval, pitch, note, chord
import glob
import numpy as np
import pandas as pd

midifile_path = glob.glob("/gdrive/My Drive/MIDI files/popM/*.mid")
midifile_path.sort(key= str.lower)

melodyLists = []
accoLists = []

melody_Beat_List=[]
acco_Beat_List=[]

barIndex = []
def setKeys(midi):
    print(str(midi.analyze('key').tonic) + midi.analyze('key').mode)
    keyTonic = midi.analyze('key').tonic
    mode = midi.analyze('key').mode
    #The pitch of the tone of the original key
    originPitch = pitch.Pitch(keyTonic)
    #set target key
    if mode == "major":
        targetPitch = pitch.Pitch('C')
        pitchInterval = interval.notesToChromatic(originPitch,targetPitch).semitones
        if pitchInterval > 6:
            pitchInterval = pitchInterval-12
        elif pitchInterval < -6:
            pitchInterval = pitchInterval+12
        print(pitchInterval)
        if keyTonic != "C":
            midi.transpose(pitchInterval ,inPlace=True)
    elif mode == "minor":
        targetPitch = pitch.Pitch('A')
        pitchInterval = interval.notesToChromatic(originPitch,targetPitch).semitones
        if pitchInterval > 6:
            pitchInterval = pitchInterval-12
        elif pitchInterval < -6:
            pitchInterval = pitchInterval+12
        print(pitchInterval)
        if keyTonic != "A":
            midi.transpose(pitchInterval ,inPlace=True)
    return midi


def parseMelody(midi):
    midi = midi.makeMeasures()
    measures = midi.recurse(classFilter=('Measure'), restoreActiveSites=False).elements
#     barIndex.append(len(measures))
    barIndex.append((len(melodyLists),len(measures)))
    for measure in measures:
        elements = []
        note_or_chord = measure.recurse(classFilter=('Note','Chord'), restoreActiveSites=False).elements
        for ele in note_or_chord:
            if isinstance(ele, note.Note):
                elements.append(str(ele.pitch))
            elif isinstance(ele, chord.Chord):
                ele = '.'.join(str(c) for c in ele.normalOrder)
                elements.append(ele)
            else:
                elements.append(str(ele.name))
        melodyLists.append(elements)
#     barIndex.append(len(melodyLists))

def parseAcco(midi):
    midi = midi.makeMeasures()
    measures = midi.recurse(classFilter=('Measure'), restoreActiveSites=False).elements
    for measure in measures:
        elements = []
        note_or_chord = measure.recurse(classFilter=('Note','Chord'), restoreActiveSites=False).elements
        for ele in note_or_chord:
            if isinstance(ele, note.Note):
                elements.append(str(ele.pitch))
            elif isinstance(ele, chord.Chord):
                ele = '.'.join(str(c) for c in ele.normalOrder)
                elements.append(ele)
            else:
                elements.append(str(ele.name))
        accoLists.append(elements)

def parseBeat(midi, list):
    midi = midi.makeMeasures(inPlace=False)
    measures = midi.recurse(classFilter=('Measure'), restoreActiveSites=False).elements
    for measure in measures:
        note_or_chord = measure.recurse(classFilter=('Note','Chord'), restoreActiveSites=False).elements
        n = [float(t.offset) for t in note_or_chord]
        list.append(n)
    return list

for file in midifile_path:
    print(file)
    midifile = midi.translate.midiFilePathToStream(file)
    midifile = setKeys(midifile)

    melody = midifile.parts[0]
    parseMelody(melody)
    melody_Beat_List = parseBeat(melody,melody_Beat_List)
    accompany = midifile.parts[1]
    parseAcco(accompany)
    acco_Beat_List = parseBeat(accompany,acco_Beat_List)

melodyLists = np.reshape(melodyLists, (len(melodyLists),1))
accoLists = np.reshape(accoLists, (len(accoLists),1))

target_MA = np.concatenate((melodyLists,accoLists),axis=1)
target_MM = np.concatenate((melodyLists[:-1],melodyLists[1:]),axis=1)

df_MA = pd.DataFrame(target_MA,columns=['melody','accompany'])
print(df_MA.head())
df_MA.to_csv('Melody_Accompany.csv')

df_MM = pd.DataFrame(target_MM,columns=['prev','next'])
print(df_MM.head())
df_MM.to_csv('Melody_Melody.csv')

df_beat_M = pd.DataFrame(melody_Beat_List)
df_beat_A = pd.DataFrame(acco_Beat_List)

df_beat_M.to_csv('beats_of_Melody.csv')
df_beat_A.to_csv('beats_of_Accompany.csv')

df_BI = pd.DataFrame(barIndex, columns=['start','length'])
print(df_BI.head())
df_BI.to_csv('bar_index.csv')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/MIDI files/popM/twins-下一站天后.mid
Bmajor
1
/gdrive/My Drive/MIDI files/popM/周杰伦-兰亭序.mid
Dmajor
-2
/gdrive/My Drive/MIDI files/popM/周杰伦-夜曲.mid
Eminor
5
/gdrive/My Drive/MIDI files/popM/周杰伦-安静.mid
B-major
2
/gdrive/My Drive/MIDI files/popM/周杰伦-彩虹.mid
Cmajor
0
/gdrive/My Drive/MIDI files/popM/周杰伦-明明就.mid
Amajor
3
/gdrive/My Drive/MIDI files/popM/周杰伦-烟花易冷.mid
Aminor
0
/gdrive/My Drive/MIDI files/popM/周杰伦-甜甜的.mid
Emajor
-4
/gdrive/My Drive/MIDI files/popM/周

### The training model is a "word-level" seq2seq model derived from the keras NMT model 
https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

Now let's parse the .csv file we just saved. 'Melody' and 'Accompany' are two different lists of the same length, representing a collection of input and output sequences.

In [0]:
import pandas as pd
import h5py
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'Melody_Accompany.csv'
lines = pd.read_table(data_path, sep=',' , header=0, names=['melody', 'accompany'], index_col=0)
    
print(lines.head())

melody_source = lines['melody'].values.tolist()
acco_source = lines['accompany'].values.tolist()

print(melody_source)
print(acco_source)

input_seq = []
target_seq = []

for f in melody_source:
    f = str(f).strip('[]')
    if f == '':
        f = 'nan'
    txt = [eval("'rest'") if t == 'nan' else eval(t) for t in f.split(',')]
    input_seq.append(txt)

for f in acco_source:
    f = str(f).strip('[]')
    if f == '':
        f = 'nan'
    txt = [eval("'rest'") if t == 'nan' else eval(t) for t in f.split(',')]
    txt.insert(0,'\t')
    txt.append('\n')
    target_seq.append(txt)
        
print(input_seq)
        
input_texts = input_seq
target_texts =  target_seq
print(target_texts)

input_words = set()
target_words = set()

for line in input_texts[: min(num_samples, len(lines) - 1)]:
    for word in line:
        if word not in input_words:
            input_words.add(word)
for line in target_texts[: min(num_samples, len(lines) - 1)]:
    for word in line:
        if word not in target_words:
            target_words.add(word)

            
input_characters = sorted(list(input_words))
target_characters = sorted(list(target_words))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('MA.h5')


                                              melody  \
0         ['E5', 'G5', 'D6', 'C6', 'G5', 'E5', 'G5']   
1      ['D5', 'G5', 'D6', 'C6', 'G5', '11.4', '0.4']   
2  ['B5', 'C6', 'B5', 'A5', 'G5', '2.5', 'E5', 'D5']   
3                    ['C5', 'G4', 'C5', 'D5', '4.7']   
4         ['E5', 'E5', 'E5', 'G5', 'D5', 'D5', 'G5']   

                                          accompany  
0  ['C3', 'G3', 'C4', 'G3', 'F2', 'C3', 'F3', 'A3']  
1  ['G2', 'D3', 'G3', 'D3', 'A2', 'E3', 'A3', 'G3']  
2                   ['F2', 'C3', 'F3', 'A3', '2.7']  
3              ['C3', 'G3', 'C4', 'C3', 'G3', 'C4']  
4              ['C3', 'G3', 'C4', 'B2', 'G3', 'B3']  
["['E5', 'G5', 'D6', 'C6', 'G5', 'E5', 'G5']", "['D5', 'G5', 'D6', 'C6', 'G5', '11.4', '0.4']", "['B5', 'C6', 'B5', 'A5', 'G5', '2.5', 'E5', 'D5']", "['C5', 'G4', 'C5', 'D5', '4.7']", "['E5', 'E5', 'E5', 'G5', 'D5', 'D5', 'G5']", "['C5', 'C5', 'C5', 'E5', 'A4']", "['F5', 'E5', 'F5', 'A5', 'E5', 'F5', 'G5']", "['D5', 'D5', 'A4', 'C5', 'D5

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


### Using the trained seq2seq model to predict the accompany for the melody(in list form).

In [0]:
from keras.models import load_model
import pandas as pd

data_path = 'bar_index.csv'
lines = pd.read_table(data_path, sep=',', index_col=0)

print(lines.head())
bar_index = lines.values.tolist()

model = load_model('MA.h5')

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        if (sampled_char != '\n'):
            decoded_sentence.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    return decoded_sentence

decoded_sentences = []
for seq_index in range(bar_index[1][0],bar_index[1][0]+bar_index[1][1]):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentences.append(decoded_sentence)
    
df_DS = pd.DataFrame(decoded_sentences)
print(df_DS.head())
df_DS.to_csv('Accompany.csv')
    

   start  length
0      0      63
1     63      79
2    142     136
3    278      99
4    377      82
   0   1    2   3   4     5     6     7     8     9     10    11    12    13  \
0  C2  G2   C3  E3  G3    E3    C3    G2  None  None  None  None  None  None   
1  F2  F3  9.0  E2  E3  8.11  None  None  None  None  None  None  None  None   
2  A2  E3   A3  C4  G2    E3    C4  None  None  None  None  None  None  None   
3  C3  G3   C4  E4  C4    G3    B3    D4    B3    G3    B3    G3    B3  None   
4  C3  G3   C4  E4  B2    G3    B3  None  None  None  None  None  None  None   

     14    15  
0  None  None  
1  None  None  
2  None  None  
3  None  None  
4  None  None  


### Generate midi files with melody and the accompaniment(in midi stream form) being generated

In [0]:
from music21 import midi, interval, pitch, note, chord, stream
import pandas as pd
from random import choice

data_path = 'Accompany.csv'
lines = pd.read_table(data_path, sep=',', index_col=0)
list_acco = lines.values.tolist()
accompany_list = []
for f in list_acco:
    txt = [t for t in f if t == t]
    txt = [t if (type(t) is str) else '%s'%t for t in txt]
    accompany_list.append(txt)
accompany_list = [t for t in accompany_list if t != []]

data_path_beats = 'beats_of_Accompany.csv'
lines_beats = pd.read_table(data_path_beats, sep=',', index_col=0)
list_beats = lines_beats.values.tolist()
beat_list = []
for f in list_beats:
    txt = [t for t in f if t == t]
    txt = [t if (type(t) is str) else '%s'%t for t in txt]
    beat_list.append(txt)
beat_list = [t for t in beat_list if t != []]
beat_list = sorted(beat_list, key= lambda i:len(i))
max_beat_seq_length = max([len(t) for t in beat_list])

for i in range(1, max_beat_seq_length + 1):
    l = [t for t in beat_list if len(t) == i]
    l = [" ".join(b) for b in l]
    locals()['beats_with_%s_note' % i] = l
    pass

midifile_path = "/gdrive/My Drive/MIDI files/popM/周杰伦-兰亭序.mid"

def setKeys(midi):
    keyTonic = midi.analyze('key').tonic
    mode = midi.analyze('key').mode
    #The pitch of the tone of the original key
    originPitch = pitch.Pitch(keyTonic)
    #set target key
    if mode == "major":
        targetPitch = pitch.Pitch('C')
        pitchInterval = interval.notesToChromatic(originPitch,targetPitch).semitones
        if pitchInterval > 6:
            pitchInterval = pitchInterval-12
        elif pitchInterval < -6:
            pitchInterval = pitchInterval+12
        if keyTonic != "C":
            midi.transpose(pitchInterval ,inPlace=True)
    elif mode == "minor":
        targetPitch = pitch.Pitch('A')
        pitchInterval = interval.notesToChromatic(originPitch,targetPitch).semitones
        if pitchInterval > 6:
            pitchInterval = pitchInterval-12
        elif pitchInterval < -6:
            pitchInterval = pitchInterval+12
        if keyTonic != "A":
            midi.transpose(pitchInterval ,inPlace=True)
    return midi

midifile = midi.translate.midiFilePathToStream(midifile_path)
midifile = setKeys(midifile)

melody = midifile.parts[0]
melody = melody.makeMeasures()

accompaniment = midifile.parts[1]
accompaniment = accompaniment.makeMeasures()

score = stream.Score()
part1 = stream.Part()
part1.insert(0,melody)
part2 = stream.Part()

measure_list = []
measures = accompaniment.recurse(classFilter=('Measure'), restoreActiveSites=True).elements
for measure in measures:
    measure.removeByNotOfClass('Measure')
    measure_list.append(measure)
measure_stream = stream.Stream(measure_list)

accompany_in_measures = []
print('measures count=',len(accompany_list))
for n in range(len(accompany_list)):
    length = len(accompany_list[n])
    beat_map = choice(locals()['beats_with_%s_note' % length])
    beat_map = [float(b) for b in beat_map.split(' ')]
    bar = []
    for ele in accompany_list[n]:
        e_index = accompany_list[n].index(ele)
        offset = beat_map[e_index]
        # pattern is a chord
        if ('.' in ele) or ele.isdigit():
            notes_in_chord = ele.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            bar.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(ele)
            new_note.offset = offset
            bar.append(new_note)
    accompany_in_measures.append(bar)


for i in range(len(measure_stream)):
    measure = measure_stream[i]
    midi_stream = stream.Stream(accompany_in_measures[i])
    measure.insert(0,midi_stream)

part2.insert(0,measure_stream)
score.insert(0,part1)
score.insert(0,part2)

score.write('midi',fp = 'test_output.mid')

[['C2', 'G2', 'C3', 'E3', 'G3', 'E3', 'C3', 'G2'], ['F2', 'F3', '9.0', 'E2', 'E3', '8.11'], ['A2', 'E3', 'A3', 'C4', 'G2', 'E3', 'C4'], ['C3', 'G3', 'C4', 'E4', 'C4', 'G3', 'B3', 'D4', 'B3', 'G3', 'B3', 'G3', 'B3'], ['C3', 'G3', 'C4', 'E4', 'B2', 'G3', 'B3'], ['G3', 'A3', 'C4', 'G4', 'B3', 'D4', 'G4'], ['A2', 'E3', 'A3', 'C4', 'D3', 'F#3', 'A3', 'C4'], ['F2', 'C3', 'A3', 'C3', '7', '7.11', 'D4'], ['C2', 'C3', '0.4.7', 'C3', 'G3', '0.4', 'D2', 'D3', '2.5.9', 'D3', 'A3', '2.5'], ['C3', 'G3', 'C4', 'G3', 'E3', 'G#3', 'B3', 'G#3'], ['C3', 'G3', 'C4', '4.7'], ['D2', 'A2', 'D3', 'F3', '9.2'], ['A2', 'E3', 'A3', 'C4', '4.9'], ['F2', 'C3', 'F3', 'C3', 'G2', 'D3', 'G3', 'D3'], ['C3', 'G3', 'C4', 'G3', 'E3', 'G#3', 'B3', 'G#3'], ['A2', 'E3', 'A3', '0.4'], ['A2', 'E3', 'C4', 'E3', 'E2', 'B2', 'G3', 'B2'], ['C3', 'G3', 'C4', 'G3', 'E3', 'G#3', 'B3', 'G#3'], ['C3', 'G3', 'C4', 'G3', 'E3', 'G#3', 'B3', 'G#3'], ['D2', 'A2', 'D3', 'F3', '9.2'], ['G2', 'G3', 'C4', '2.7'], ['F2', 'C3', 'F3', 'C3', 'G2',

'test_output.mid'